In [3]:
import joblib

model = joblib.load('brain_cluster.sav')
tumorLabel = 0

In [12]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

test_path = 'test_images'
test_names = os.listdir(test_path)

for test_name in test_names:
    if test_name == "Thumbs.db":
        continue
    img_path = os.path.join(test_path, test_name)
    img = cv2.imread(img_path, 0)
    imgOriginal = cv2.imread(img_path)
    height, width = img.shape
    imgFlatten = img.reshape(height*width, 1)
    labels = model.predict(imgFlatten)
    labels2D = labels.reshape(height, width)
    mask = (labels2D==tumorLabel)
    tumorExtracted = np.bitwise_and(mask, img)
    contours, hiearachy = cv2.findContours(tumorExtracted, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for index, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        
        if area > 1000:
            cv2.drawContours(imgOriginal, [cnt], -1, (0, 255, 255), 2)
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(imgOriginal, (x, y), (x+w, y+h), (0,255,0), 2)
            #cv2.rectangle(imgOriginal, (x, y), (x+120, y-40), (0,255,0), -1)
            #cv2.putText(imgOriginal, "Detected", (x+10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('Live', imgOriginal)
    cv2.waitKey(3000)
    cv2.destroyAllWindows()